<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-modal-notebook/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Modal notebooks!

Write Python code and collaborate in real time. Your code runs in Modal's
**serverless cloud**, and anyone in the same workspace can join.

This notebook comes with some common Python libraries installed. Run
cells with `Shift+Enter`.

## How do you load four files, __init__.py, hello.py, open.py and helpers.py from github and use them in a Modal Jupyter notebook?


In [ ]:
import os
import sys
import modal

# Create 'utils' directory if it doesn't exist
if not os.path.exists('utils'):
    os.makedirs('utils')

# Create 'utils' directory if it doesn't exist
if not os.path.exists('agents'):
    os.makedirs('agents')

# Check and download __init__.py
init_file = 'utils/__init__.py'
if not os.path.exists(init_file):
    print(f"Downloading {init_file}...")
    ! wget -P utils/ https://raw.githubusercontent.com/NormLorenz/ai-llm-modal-notebook/refs/heads/main/utils/__init__.py

# Check and download helpers.py
helpers_file = 'utils/helpers.py'
if not os.path.exists(helpers_file):
    print(f"Downloading {helpers_file}...")
    ! wget -P utils/ https://raw.githubusercontent.com/NormLorenz/ai-llm-modal-notebook/refs/heads/main/utils/helpers.py

# Check and download hello.py
hello_file = 'agents/hello.py'
if not os.path.exists(hello_file):
    print(f"Downloading {hello_file}...")
    ! wget -P agents/ https://raw.githubusercontent.com/NormLorenz/ai-llm-modal-notebook/refs/heads/main/agents/hello.py

# Check and download openai_file.py
openai_file = 'agents/openai.py'
if not os.path.exists(openai_file):
    print(f"Downloading {openai_file}...")
    ! wget -P agents/ https://raw.githubusercontent.com/NormLorenz/ai-llm-modal-notebook/refs/heads/main/agents/openai.py

# Setup the pathing
sys.path.append('/content/utils')
sys.path.append('/content/agents')

In [ ]:
# run the util.helpers methods locally
from utils.helpers import log, greet

log("Notebook started")
greet("Alice")

In [ ]:
# run the the agents.hello methods both locally and ephemeral on a Modal Debian box
from agents.hello import app, hello, hello_europe

# try out the hello agent locally
with app.run():
    reply = hello.local()
display(reply)

# try out the hello agent remotely
with app.run():
    reply = hello.remote()
display(reply)

# try out the hello agent in europe
with app.run():
    reply = hello_europe.remote()
display(reply)

In [ ]:
# # run the the agents.openai method both locally and ephemeral on a Modal Debian box
# from agents.openai import app, gpt_mini

# # try out the openai agent remotely
# with app.run():
#     reply = gpt_mini.remote("What is the capital of Germany?")
# display(reply)

In [ ]:
import openai
import modal
import os
from modal import App, Image

# Setup
app = modal.App("openai-ephemeral-demo")
image = Image.debian_slim().pip_install("openai")
secrets = [modal.Secret.from_name("openai-api-key")]

# Constants
GPU = "T4"

# Create an ephemeral function
@app.function(image=image, secrets=secrets, gpu=GPU)
def openai_ephemeral(prompt: str):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content

In [ ]:
# Run remotely on a Modal Debian box
with modal.enable_output():
    with app.run():
        # result = openai_ephemeral.remote("Hello from Modal ephemeral function!")
        result = openai_ephemeral.remote("What is the capital of Germany?")
result